In [1]:
import numpy
import bempp.api
from bempp.api.operators.boundary import sparse, laplace
from scipy.sparse.linalg import gmres

In [2]:
def rhs_fun(x, n, domain_index,result):
    global phi_static
    result[:] = phi_static
    
def generate_grid(filename):
    
    face = open(filename+'.face','r').read()
    vert = open(filename+'.vert','r').read()

    faces = numpy.vstack(numpy.char.split(face.split('\n')[0:-1]))[:,:3].astype(int) - 1
    verts = numpy.vstack(numpy.char.split(vert.split('\n')[0:-1]))[:,:3].astype(float)

    grid = bempp.api.grid_from_element_data(verts.transpose(), faces.transpose())
    
    N = grid.leaf_view.entity_count(0)
    elements = list(grid.leaf_view.entity_iterator(0))
    area = numpy.zeros(N)

    # remove zero areas
    for i in range(N):
        area[i] = elements[i].geometry.volume

    area_nonzero = numpy.where(area>1e-12)[0]
    
    faces_nonzero = faces[area_nonzero,:]
    
    grid = bempp.api.grid_from_element_data(verts.transpose(), faces_nonzero.transpose())
    
    return grid

def energy_mol(mesh_diel, mesh_stern, phi_static, eps_s):

    grid_diel = generate_grid(mesh_diel)
    grid_stern = generate_grid(mesh_stern)

    space_diel = bempp.api.function_space(grid_diel, "DP", 0)
    space_stern = bempp.api.function_space(grid_stern, "DP", 0)
    
    N_diel = grid_diel.leaf_view.entity_count(0)
    N_stern = grid_stern.leaf_view.entity_count(0)
    
    elements_d = list(grid_diel.leaf_view.entity_iterator(0))
    elements_s = list(grid_stern.leaf_view.entity_iterator(0))

    phis_grid_fun = bempp.api.GridFunction(space_diel, fun=rhs_fun)

    rhs = numpy.concatenate([eps_s*phis_grid_fun.coefficients, 
                      numpy.zeros(N_stern)])
    

    M11   = laplace.single_layer(space_diel, space_diel, space_diel) 
    M12   = laplace.single_layer(space_stern, space_diel, space_diel) 
    M21   = laplace.single_layer(space_diel, space_stern, space_stern)
    M22   = laplace.single_layer(space_stern, space_stern, space_stern) 

    blocked = bempp.api.BlockedOperator(2, 2)
    blocked[0,0] = M11
    blocked[0,1] = M12
    blocked[1,0] = M21
    blocked[1,1] = M22
    op_discrete = blocked.strong_form()
    
    sigma, info = gmres(op_discrete, rhs, tol=1e-5, maxiter=500, restart = 1000)
    
    if info>0:
        print 'Not converged, %i iterations'%info
    elif info<0:
        print 'Solver diverges'
    
    sigma_d = sigma[:N_diel]
    sigma_s = sigma[N_diel:]

    '''
    qe = 1.60217662e-19
    Rw = 1.4
    Na = 6.0221409e23
    water_charge = 0.83*qe/(numpy.pi*Rw**2) * Na/(4.184*1000)
    test_charge = numpy.where(abs(sigma_d)>water_charge)[0]
    #print abs(sigma_d)
    if len(test_charge)>0:
        print 'Unphysical charge in dielectric surface'
        print len(test_charge)
        
    
    test_charge = numpy.where(abs(sigma_s)>water_charge)[0]
    if len(test_charge)>0:
        print 'Unphysical charge in stern surface'
        print len(test_charge)
    '''
    area_d = numpy.zeros(N_diel)
    for i in range(N_diel):
        area_d[i] = elements_d[i].geometry.volume
    
    energy = 0.5*numpy.sum(area_d*sigma_d*phi_static)

    return energy

In [3]:
phi_static = 10. #kcal/mol/e
phi_static *= 4.184 # kJ/mol/e

qe = 1.60217662e-19
Na = 6.0221409e23
eps_0 = 8.854187817e-12
eps_s = 1.#80.

conv_factor = (1000*eps_0)/(qe**2*Na*1e10)

problem_folder = 'mobley_test/'
namefile = 'actually_all_names.txt'
num_lines = sum(1 for line in open(problem_folder+namefile))
energy = numpy.zeros(num_lines)
it = 0
for line in file(problem_folder+namefile):
    mol = line[:-1]

    mesh_stern = problem_folder+mol+'/surf_d02_stern'
    mesh_diel = problem_folder+mol+'/surf_d02'

    try: 
        e = energy_mol(mesh_diel, mesh_stern, phi_static, eps_s)
    except:
        print 'Something went wrong'
        e = 0
            
    ekJ = e*conv_factor # kJ/mol
    ekcal = ekJ/4.184   # kcal/mol
    
    energy[it] = ekcal
    
    print 'Run %i for '%it + mol + '. Energy = %1.3f kcal/mol'%energy[it]
    
    it += 1
    
numpy.savetxt('energy_bem.txt', energy)

Run 0 for 1112_tetrachloroethane. Energy = 1.700 kcal/mol
Run 1 for 111_trichloroethane. Energy = 1.573 kcal/mol
Run 2 for 111_trifluoro_222_trimethoxyethane. Energy = 2.037 kcal/mol
Run 3 for 111_trifluoropropan_2_ol. Energy = 1.543 kcal/mol
Run 4 for 111_trimethoxyethane. Energy = 1.914 kcal/mol
Run 5 for 1122_tetrachloroethane. Energy = 1.688 kcal/mol
Run 6 for 112_trichloro_122_trifluoroethane. Energy = 1.715 kcal/mol
Run 7 for 112_trichloroethane. Energy = 1.545 kcal/mol
Run 8 for 11_diacetoxyethane. Energy = 2.243 kcal/mol
Run 9 for 11_dichloroethane. Energy = 1.416 kcal/mol
Run 10 for 11_dichloroethene. Energy = 1.364 kcal/mol
Run 11 for 11_diethoxyethane. Energy = 2.207 kcal/mol
Run 12 for 11_difluoroethane. Energy = 1.223 kcal/mol
Run 13 for 1234_tetrachlorobenzene. Energy = 2.130 kcal/mol
Run 14 for 1235_tetrachlorobenzene. Energy = 2.148 kcal/mol
Run 15 for 123_trichlorobenzene. Energy = 1.987 kcal/mol
Run 16 for 123_trimethylbenzene. Energy = 2.084 kcal/mol
Run 17 for 1245_

Run 149 for 3_acetylpyridine. Energy = 1.894 kcal/mol
Run 150 for 3_chloroaniline. Energy = 1.796 kcal/mol
Run 151 for 3_chlorophenol. Energy = 1.768 kcal/mol
Run 152 for 3_chloroprop_1_ene. Energy = 1.403 kcal/mol
Run 153 for 3_chloropyridine. Energy = 1.637 kcal/mol
Run 154 for 3_cyanophenol. Energy = 1.846 kcal/mol
Run 155 for 3_cyanopyridine. Energy = 1.713 kcal/mol
Run 156 for 3_ethylphenol. Energy = 2.021 kcal/mol
Run 157 for 3_ethylpyridine. Energy = 1.893 kcal/mol
Run 158 for 3_formylpyridine. Energy = 1.688 kcal/mol
Run 159 for 3_hydroxybenzaldehyde. Energy = 1.818 kcal/mol
Run 160 for 3_methoxyaniline. Energy = 1.914 kcal/mol
Run 161 for 3_methoxyphenol. Energy = 1.884 kcal/mol
Run 162 for 3_methyl_1h_indole. Energy = 2.034 kcal/mol
Run 163 for 3_methyl_but_1_ene. Energy = 1.664 kcal/mol
Run 164 for 3_methylbut_1_ene. Energy = 1.664 kcal/mol
Run 165 for 3_methylbutan_1_ol. Energy = 1.727 kcal/mol
Run 166 for 3_methylbutan_2_one. Energy = 1.733 kcal/mol
Run 167 for 3_methylbut

Run 303 for dimethyl_sulfoxide. Energy = 1.359 kcal/mol
Run 304 for dimethylamine. Energy = 1.180 kcal/mol
Run 305 for ethanamide. Energy = 1.216 kcal/mol
Run 306 for ethane. Energy = 1.116 kcal/mol
Run 307 for ethanethiol. Energy = 1.278 kcal/mol
Run 308 for ethanol. Energy = 1.154 kcal/mol
Run 309 for ethene. Energy = 0.999 kcal/mol
Run 310 for ethyl_acetate. Energy = 1.708 kcal/mol
Run 311 for ethyl_benzoate. Energy = 2.311 kcal/mol
Run 312 for ethyl_butanoate. Energy = 2.146 kcal/mol
Run 313 for ethyl_formate. Energy = 1.439 kcal/mol
Run 314 for ethyl_hexanoate. Energy = 2.597 kcal/mol
Run 315 for ethyl_pentanoate. Energy = 2.368 kcal/mol
Run 316 for ethyl_phenyl_ether. Energy = 2.086 kcal/mol
Run 317 for ethyl_propanoate. Energy = 1.924 kcal/mol
Run 318 for ethylamine. Energy = 1.187 kcal/mol
Run 319 for ethylbenzene. Energy = 1.954 kcal/mol
Run 320 for fluorene. Energy = 2.385 kcal/mol
Run 321 for fluorobenzene. Energy = 1.608 kcal/mol
Run 322 for fluoromethane. Energy = 0.893 kc

Run 462 for phenyl_trifluoroethyl_ether. Energy = 2.237 kcal/mol
Run 463 for piperazine. Energy = 1.519 kcal/mol
Run 464 for piperidine. Energy = 1.620 kcal/mol
Run 465 for prop_2_en_1_ol. Energy = 1.304 kcal/mol
Run 466 for propan_1_ol. Energy = 1.389 kcal/mol
Run 467 for propan_2_ol. Energy = 1.385 kcal/mol
Run 468 for propane. Energy = 1.331 kcal/mol
Run 469 for propanenitrile. Energy = 1.355 kcal/mol
Run 470 for propanoic_acid. Energy = 1.403 kcal/mol
Run 471 for propanone. Energy = 1.344 kcal/mol
Run 472 for propene. Energy = 1.251 kcal/mol
Run 473 for propionaldehyde. Energy = 1.334 kcal/mol
Run 474 for propyne. Energy = 1.191 kcal/mol
Run 475 for pyrene. Energy = 2.583 kcal/mol
Run 476 for pyridine. Energy = 1.468 kcal/mol
Run 477 for pyrrole. Energy = 1.317 kcal/mol
Run 478 for pyrrolidine. Energy = 1.456 kcal/mol
Run 479 for quinoline. Energy = 1.968 kcal/mol
Run 480 for sec_butylbenzene. Energy = 2.285 kcal/mol
Run 481 for styrene. Energy = 1.875 kcal/mol
Run 482 for tefluran

In [ ]:
mesh_stern = 'mobley_test/1112_tetrachloroethane/surf_d02_stern'
mesh_diel = 'mobley_test/1112_tetrachloroethane/surf_d02'

phi_static = 40. #kJ/mol/e


e = energy_mol(mesh_diel, mesh_stern, phi_static, eps_s)
ekj = e / ((qe**2*Na*1e10)/(1000*eps_0))
ekcal = ekj/4.184
print e, e / ((qe**2*Na*1e10)/(1000*eps_0)), e/(1000*4.184) #kJ/mol
print ekcal

In [ ]:
F = 1000*eps_0/(qe**2*Na*1e10)
1/(4*numpy.pi*F*4.184)
9081457.48434/(332.063*4*numpy.pi)
(qe**2*Na*1e10)/(eps_0)

In [ ]:
1.54